In [1]:
import aqnsim

In [2]:
# 5 Players
# Each player receives 2 partial bell states, and 8 |+> states

NUM_PLAYERS = 5
NUM_ROUNDS = 2

In [4]:
class Player(aqnsim.Node):
    def __init__(self, sim_context: aqnsim.SimulationContext, name: str):
        super().__init__(
            sim_context=sim_context,
            ports=["distributor"],
            name=name
        )
        self.data_collector.register_attribute(self.name)
        self.bit_vector = []

        self.qmemory = aqnsim.QMemory(
            sim_context=self.sim_context,
            n=1,
            ports=["distributor"], 
            name=f"QMemory-{name}",
        )

    @aqnsim.process
    def measure_qubit(self):
        meas_result = yield self.qmemory.measure(0)
        self.bit_vector.append(meas_result)
        self.data_collector.update_attribute(self.name, self.bit_vector)
        


class Distributor(aqnsim.Node):
    def __init__(self, sim_context: aqnsim.SimulationContext, name: str):
        super().__init__(
            sim_context=sim_context,
            ports=[f"player{i}" for i in range(NUM_PLAYERS)],
            name=name
        )
        self.data_collector.register_attribute(self.name)

        self.qmemory = aqnsim.QMemory(
            sim_context=self.sim_context,
            n=NUM_PLAYERS,
            ports=[f"player{i}" for i in range(NUM_PLAYERS)], 
            name=f"QMemory-{name}",
        )
        for i in range(NUM_PLAYERS):
            self.qmemory.ports[f"player{i}"].forward_output_to_output(self.ports[f"player{i}"])  # Is this correct?
            

    # Try using circuits instead looking at the quantum networks and protocols

    @aqnsim.process
    def create_epr_pair(self, q1_idx, q2_idx):
        yield self.qmemory.operate(aqnsim.ops.H, qpos=q1_idx)
        yield self.qmemory.operate(aqnsim.ops.X, qpos=q2_idx)
        yield self.qmemory.operate(aqnsim.ops.CNOT, qpos=[q1_idx, q2_idx])

    @aqnsim.process
    def create_plus_state(self, q_idx):
        yield self.qmemory.operate(aqnsim.ops.H, qpos=q_idx)

In [5]:
SEC = aqnsim.SECOND
OP_DELAYS = {aqnsim.H: 1 * SEC, aqnsim.X: 1 * SEC, aqnsim.Z: 1 * SEC, aqnsim.CNOT: 1 * SEC}
MEAS_DELAY = 1 * SEC
PHI_PLUS_STATE_DENSITY = aqnsim.BELL_STATES_DENSITY["phi_plus"]
QSOURCE_STATE_DISTRIBUTION = [(1, PHI_PLUS_STATE_DENSITY)]
QSOURCE_NOISE_MODEL = None
QUANTUM_CHANNEL_DELAY = 1 * SEC
QUANTUM_CHANNEL_NOISE = 0.0

In [6]:
class DistributorProtocol(aqnsim.NodeProtocol):
    def __init__(self, sim_context: aqnsim.SimulationContext, node: aqnsim.Node, name: str = None):
        super().__init__(sim_context=sim_context, node=node, name=name)

        self.distributor = node

    @aqnsim.process
    def run(self):
        current_round = 0
        while (current_round < NUM_ROUNDS):
            for j in range(1,NUM_PLAYERS):
                bell_reciever_1_idx = 0
                bell_reciever_2_idx = 1 + (j % (NUM_PLAYERS - 1))
                yield self.distributor.create_epr_pair(bell_reciever_1_idx, bell_reciever_2_idx)
                self.distributor.qmemory.positions[bell_reciever_1_idx].pop_replace(f"player{bell_reciever_1_idx}")
                self.distributor.qmemory.positions[bell_reciever_2_idx].pop_replace(f"player{bell_reciever_2_idx}")
                for k in range(1,NUM_PLAYERS):
                    if k == bell_reciever_2_idx:
                        continue
                    yield self.distributor.create_plus_state(k)
                    self.distributor.qmemory.positions[k].pop_replace(f"player{k}")
                yield self.wait(1)  # TODO: Can parameterize wait time
            current_round += 1
            self.sim_context.simlogger.info(f"Finished round {current_round}")

class PlayerProtocol(aqnsim.NodeProtocol):
    def __init__(self, sim_context: aqnsim.SimulationContext, node: aqnsim.Node, name: str = None):
        super().__init__(sim_context=sim_context, node=node, name=name)

        self.player = node
        self.player.ports["distributor"].add_rx_input_handler(
            handler=lambda msg: self.quantum_port_source_handler(msg=msg)
        )

        # self._meas_signal_name = "READY TO MEASURE QUBIT"
        # self.add_signal(self._meas_signal_name)

    @aqnsim.process
    def quantum_port_source_handler(self, msg: aqnsim.Qubit):
        
        self.sim_context.simlogger.info("qport handled")
        
        if isinstance(msg, aqnsim.Qubit):
            self.player.qmemory.positions[0].put(qubit=msg)
            yield self.player.measure_qubit()  # Measures and stores in player's BV

    # @aqnsim.process
    # def run(self):
    #     while(True):
    #         # if signal recieved
    #         if ()
    #         yield self.player.measure_qubit()
            




In [7]:
def setup_network(sim_context: aqnsim.SimulationContext) -> aqnsim.Network:
    alice = Player(sim_context = sim_context, name="Alice")
    bob = Player(sim_context = sim_context, name="Bob")
    charlie = Player(sim_context = sim_context, name="Charlie")
    david = Player(sim_context = sim_context, name="David")
    esther = Player(sim_context = sim_context, name="Esther")

    players = [alice, bob, charlie, david, esther]

    distributor = Distributor(sim_context = sim_context, name="ent_src")

    network = aqnsim.Network(sim_context=sim_context, nodes=[distributor]+players)

    # Setup the links

    qlink_es_alice = aqnsim.QuantumLink(
        sim_context = sim_context,
        delay = QUANTUM_CHANNEL_DELAY,
        noise = QUANTUM_CHANNEL_NOISE,
        name="Q_Link_Alice_Source"
        
    )

    qlink_es_bob = aqnsim.QuantumLink(
        sim_context = sim_context,
        delay = QUANTUM_CHANNEL_DELAY,
        noise = QUANTUM_CHANNEL_NOISE,
        name="Q_Link_Bob_Source"
        
    )

    qlink_es_charlie = aqnsim.QuantumLink(
        sim_context = sim_context,
        delay = QUANTUM_CHANNEL_DELAY,
        noise = QUANTUM_CHANNEL_NOISE,
        name="Q_Link_Charlie_Source"
        
    )

    qlink_es_david = aqnsim.QuantumLink(
        sim_context = sim_context,
        delay = QUANTUM_CHANNEL_DELAY,
        noise = QUANTUM_CHANNEL_NOISE,
        name="Q_Link_David_Source"
        
    )

    qlink_es_esther = aqnsim.QuantumLink(
        sim_context = sim_context,
        delay = QUANTUM_CHANNEL_DELAY,
        noise = QUANTUM_CHANNEL_NOISE,
        name="Q_Link_Esther_Source"
        
    )

    network.add_link(qlink_es_alice, distributor, alice, "player0", "distributor")
    network.add_link(qlink_es_bob, distributor, bob, "player1", "distributor")
    network.add_link(qlink_es_charlie, distributor, charlie, "player2", "distributor")
    network.add_link(qlink_es_david, distributor, david, "player3", "distributor")
    network.add_link(qlink_es_esther, distributor, esther, "player4", "distributor")

    DistributorProtocol(sim_context = sim_context, node = distributor)
    for player in players:
        PlayerProtocol(sim_context = sim_context, node = player)
        
    return network

In [8]:
run_simulation = aqnsim.generate_run_simulation_fn(
    setup_sim_fn=setup_network, logging_level=20, log_to_file=False
)
output = run_simulation()
print(output)


(0 s): INFO - DistributorProtocol ready!
(0 s): INFO - PlayerProtocol ready!
(0 s): INFO - PlayerProtocol ready!
(0 s): INFO - PlayerProtocol ready!
(0 s): INFO - PlayerProtocol ready!
(0 s): INFO - PlayerProtocol ready!
(1.0 s): INFO - qport handled
(1.0 s): INFO - qport handled
(1.0 s): INFO - qport handled
(1.0 s): INFO - qport handled
(1.0 s): INFO - qport handled
(2.0 s): INFO - qport handled
(2.0 s): INFO - qport handled
(2.0 s): INFO - qport handled
(2.0 s): INFO - qport handled
(2.0 s): INFO - qport handled
(3.0 s): INFO - qport handled
(3.0 s): INFO - qport handled
(3.0 s): INFO - qport handled
(3.0 s): INFO - qport handled
(3.0 s): INFO - qport handled
(4.0 s): INFO - qport handled
(4.0 s): INFO - qport handled
(4.0 s): INFO - qport handled
(4.0 s): INFO - qport handled
(4 s): INFO - Finished round 1
(4.0 s): INFO - qport handled
(5.0 s): INFO - qport handled
(5.0 s): INFO - qport handled
(5.0 s): INFO - qport handled
(5.0 s): INFO - qport handled
(5.0 s): INFO - qport handle

{'Alice': [([0, 1, 0, 0, 0, 0, 0, 0], 1.0), ([0, 1, 0, 0, 0, 0, 0, 0], 2.0), ([0, 1, 0, 0, 0, 0, 0, 0], 3.0), ([0, 1, 0, 0, 0, 0, 0, 0], 4.0), ([0, 1, 0, 0, 0, 0, 0, 0], 5.0), ([0, 1, 0, 0, 0, 0, 0, 0], 6.0), ([0, 1, 0, 0, 0, 0, 0, 0], 7.0), ([0, 1, 0, 0, 0, 0, 0, 0], 8.0)], 'Bob': [([0, 0, 1, 1, 0, 0, 1, 1], 1.0), ([0, 0, 1, 1, 0, 0, 1, 1], 2.0), ([0, 0, 1, 1, 0, 0, 1, 1], 3.0), ([0, 0, 1, 1, 0, 0, 1, 1], 4.0), ([0, 0, 1, 1, 0, 0, 1, 1], 5.0), ([0, 0, 1, 1, 0, 0, 1, 1], 6.0), ([0, 0, 1, 1, 0, 0, 1, 1], 7.0), ([0, 0, 1, 1, 0, 0, 1, 1], 8.0)], 'Charlie': [([1, 0, 0, 1, 1, 0, 0, 1], 1.0), ([1, 0, 0, 1, 1, 0, 0, 1], 2.0), ([1, 0, 0, 1, 1, 0, 0, 1], 3.0), ([1, 0, 0, 1, 1, 0, 0, 1], 4.0), ([1, 0, 0, 1, 1, 0, 0, 1], 5.0), ([1, 0, 0, 1, 1, 0, 0, 1], 6.0), ([1, 0, 0, 1, 1, 0, 0, 1], 7.0), ([1, 0, 0, 1, 1, 0, 0, 1], 8.0)], 'David': [([0, 0, 0, 0, 0, 1, 0, 0], 1.0), ([0, 0, 0, 0, 0, 1, 0, 0], 2.0), ([0, 0, 0, 0, 0, 1, 0, 0], 3.0), ([0, 0, 0, 0, 0, 1, 0, 0], 4.0), ([0, 0, 0, 0, 0, 1, 0, 0], 5.0),